In [1]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore
#get_ipython().magic(u'matplotlib inline')
import cPickle
from multiprocessing import Pool
from sklearn import preprocessing

No handlers could be found for logger "gensim.models.word2vec"


In [16]:
def dataframe2corpus(df,name_file):
    '''
    df : dataframe containing the data
    name_file : name of file saved containing the dictionnary for the lda model
    '''
    non_num_columns=['AffinityCodeId','SelectedPackage','NameOfPolicyProduct','SCID']
    le = preprocessing.LabelEncoder()
    for column in non_num_columns:
        df[column]=le.fit_transform(df[column].as_matrix())
    dropped_columns=['CustomerMD5Key','Unnamed: 0']
    all_columns=df.columns.tolist()
    for column_name in dropped_columns:
        all_columns.remove(column_name)

    reduced_df=df[all_columns]
    
    # List of column names with categorical variables
    categ_vars=[]
    for column in reduced_df.columns:
        if len(np.unique(reduced_df[column]))<50:    # we don't keep columns with too many different values
            categ_vars+=[column]

# Encode the data as a bag of words to feed gensim LDA
    equiv_texts=[]
    all_cat=reduced_df[categ_vars].as_matrix().tolist()
    for row in all_cat:
        text_row=[str(categ_vars[i])+':'+str(word) for (i,word) in enumerate(row)]
        equiv_texts+=[text_row]
    
    
    #dictionary = corpora.Dictionary(equiv_texts)
    #dictionary.save('topics_'+str(name_file)+'.dict') # dictionnary id2word
    dictionary=corpora.Dictionary.load('topics_alex.dict')
    
    
    
    corpus = [dictionary.doc2bow(text) for text in equiv_texts]
    corpora.MmCorpus.serialize('topics_'+str(name_file)+'.mm', corpus) # serialized dictionnary (for fast lda)
    return



In [17]:
#import time

n_topics=5
#t1=time.time()

name_file='priv'

In [18]:
X_test=pd.read_csv('X_priv.csv')
#X_train=pd.read_csv('X_priv.csv')

df=X_test
#del X_test

dataframe2corpus(df,name_file)
lenX=np.shape(df.as_matrix())[0]
#del df


#t2=time.time() # step 1 done : stored the vocabulary in the data
#print(t2-t1)

In [19]:
# step 2 : 
#t2=time.time()


lda=cPickle.load(open('lda_alex_'+str(n_topics)+'_topics.p','rb')) # load lda model trained on the big corpus

mm_test = corpora.MmCorpus('topics_'+str(name_file)+'.mm')
id2word = corpora.Dictionary.load('topics_alex.dict')
   
#print('updating')
#lda.update(mm)
#print('ended updating')    
    


ldamm=lda[mm_test]

del mm_test,lda # saving memory

L_indices=list(range(lenX)) ##########


def transform_vocab(idx_doc_bow):
    global n_topics,ldamm
    bow_converted=ldamm[idx_doc_bow]
    local_array=[0]*(n_topics+1)
    for feat_tuple in bow_converted:
        local_array[feat_tuple[0]+1]=feat_tuple[1]
    return local_array
        
# Compute the feature vector :
p=Pool(10)
a=p.map(transform_vocab,L_indices)

cPickle.dump(a,open('sauvegarde_qui_sauve_la_vie.p','wb'))
topic_features_df=pd.DataFrame(a)
del a
topic_features_df=topic_features_df[topic_features_df.columns[1:]]
topic_features_df.columns=['lda_feature_'+str(i) for i in range(len(topic_features_df.columns))]
topic_features_df.to_csv('lda_features_'+str(n_topics)+'_'+str(name_file)+'_topics_df.csv',index=None)
#compute_lda_feature(n_topics,name_file)
#t3=time.time()
#print(t2-t1)
#print(t3-t2)

In [21]:
topic_features_df

,lda_feature_0,lda_feature_1,lda_feature_2,lda_feature_3,lda_feature_4
0,0.010324,0.010248,0.718463,0.250728,0.010238
1,0.540685,0.010182,0.198222,0.240622,0.010289
2,0.010183,0.010210,0.223609,0.010228,0.745770
3,0.650474,0.155000,0.174075,0.010253,0.010198
4,0.010263,0.093423,0.228808,0.657270,0.010236
5,0.906562,0.010273,0.062625,0.010320,0.010219
6,0.010211,0.010262,0.208974,0.010442,0.760110
7,0.010154,0.544102,0.205634,0.229895,0.010216
8,0.010394,0.010280,0.119963,0.557514,0.301849
9,0.010313,0.660569,0.152882,0.165997,0.010239
